In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [2]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/1721030403_LeKhaHieu/project/'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Mounted at /content/drive


[]

In [3]:
# 1. DATA

In [4]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/1721030403_LeKhaHieu/data_face_features.pickle", mode="rb"))

In [5]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.04440678  0.21475683  0.01395946 ... -0.0886332   0.06150215
   -0.00736863]]

 [[ 0.04842703  0.20101416  0.0238631  ... -0.08374722  0.07649732
    0.00379119]]

 [[ 0.05084311  0.17329712 -0.01429705 ... -0.06301611  0.11193115
    0.02131533]]

 ...

 [[ 0.07341399  0.06518976 -0.10309623 ... -0.0834958   0.08147261
    0.0745829 ]]

 [[ 0.07607125  0.05514786 -0.07174882 ... -0.02091191  0.10496895
    0.12454553]]

 [[ 0.15304564 -0.07706067 -0.08126479 ... -0.11379657  0.00884555
    0.00933749]]]
['1721031561_NguyenDucHieu' '1721031561_NguyenDucHieu'
 '1721031561_NguyenDucHieu' ... '162000309_LeHoangQuan'
 '162000309_LeHoangQuan' '162000309_LeHoangQuan']


In [6]:
x.shape

(4694, 1, 128)

In [7]:
y.shape

(4694,)

In [8]:
x = x.reshape(-1, 128)

In [9]:
x.shape

(4694, 128)

In [10]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, random_state=1)

In [11]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4224, 128), (470, 128), (4224,), (470,))

In [12]:
# 2. Train machine learning

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [14]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [15]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [16]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.73
Accurency Test = 0.69
F1 Score Train = 0.52
F1 Score Test = 0.52


In [17]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [18]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.86
F1 Score Train = 0.84
F1 Score Test = 0.79


In [19]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [20]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.80
F1 Score Train = 1.00
F1 Score Test = 0.73


In [21]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [22]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [23]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.97
Accurency Test = 0.86
F1 Score Train = 0.97
F1 Score Test = 0.77


In [24]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [25]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   9.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=  10.3s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.9s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   8.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   8.4s
[

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [26]:
model_best_estimator = model_grid.best_estimator_

In [27]:
model_grid.best_score_

0.8517992424242425

In [28]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/1721030403_LeKhaHieu/ml_face_person_identity.pkl", mode='wb'))